# Cvičení 9 - Neuronové sítě - PyTorch, dávkové učení

V tomto cvičení si nejprve představíme knihovnu PyTorch, pak v ní zkonstruujeme analogický model jako na minulém cvičení a natrénujeme ho.

In [ ]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import colormaps
from matplotlib import colors
cmap = colors.LinearSegmentedColormap(
    "red_blue_classes",
    {
        "red": [(0, 1, 1), (1, 0.7, 0.7)],
        "green": [(0, 0.7, 0.7), (1, 0.7, 0.7)],
        "blue": [(0, 0.7, 0.7), (1, 1, 1)],
    },
)
colormaps.register(cmap)

# Knihovna PyTorch

Předchozí způsob, kterým jsme neuronovou síť zkonstruovali se samozřejmě v reálu nikdy nepoužívá.

Pro konstrukci a trénování neuronových sítí se používají knihovny, pomocí kterých je nejenom možné síť snadno zkonstruovat, ale především ji také snadno a efektivně trénovat. Hlavní princip je založen na tom, že tyto knihovny umí automaticky počítat gradient.

Nejpoužívanější knihovny pro python jsou [TensorFlow](https://www.tensorflow.org/) a [PyTorch](https://pytorch.org/).
Obě jsou v mnoha věcech podobné. V našem případě budeme dále používat knihovnu `PyTorch`.
Detailní tutoriály k mnoha případům použití najdete zde: [oficiální tutoriály](https://pytorch.org/tutorials/index.html).

Začneme importem základních prvků.

In [ ]:
import torch
from torch.utils.data import DataLoader

## Tenzory a práce s nimi v PyTorch

Nejprve se podívejme na to, co se vlastně při standardním využití děje na pozadí - manipulace s tenzory a automatické počítání gradientu.

Tenzor je vlastně název pro $n$ rozměrné pole - tj. jako `ndarray` v numpy.

### Vytváření tenzorů
Tenzory můžeme vytvářet různými způsoby:
* Z existujících Python nebo numpy polí.
* Se všemi složkami stejnými při zadaných rozměrech.
* S náhodně vygenerovanými složkami při zadaných rozměrech.
* Jako speciální matice - např. jednotkové.

In [ ]:
# Převod numpy nebo python matice na konstantní tenzor
a = torch.tensor([[1, 2], [3,4]])
print(a)

np_array = np.array(a)
b = torch.from_numpy(np_array)
print(b)

In [ ]:
x_ones = torch.ones_like(a) # retains the properties of x_data
print(f"Tensor samých jedniček o stejném rozměru jako existující tenzor: \n {x_ones} \n")

x_rand = torch.rand_like(a, dtype=torch.float) # overrides the datatype of x_data
print(f"Tenzor s náhodnými složkami o stejném rozměru jako existující tenzor: \n {x_rand} \n")

In [ ]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Náhodný Tensor se zadanými rozměry: \n {rand_tensor} \n")
print(f"Tensor jedniček se zadanými rozměry: \n {ones_tensor} \n")
print(f"Tensor nul se zadanými rozměry: \n {zeros_tensor}")

### Atributy tenzorů
Tenzory podporují klasické atributy jako `shape` a `dtype`. Atribut `device` navíc umí vrátit na kterém zařízení (CPU nebo GPU) se tenzor nachází.

In [ ]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

### Nastavení datového typu tenzoru

Při vytváření nového tenzoru nebo u existujícího je možné nastavit/změnit datový typ jeho složek.

Nejčastěji používané typu jsou `torch.float` (= `torch.float32`) a `torch.long` (=`torch.int64`).

In [ ]:
a = torch.tensor([[1, 2], [3,4]])
print(a, a.dtype)

a = torch.tensor([[1, 2], [3,4]], dtype = torch.float)
print(a, a.dtype)

a = a.type(torch.long)
print(a, a.dtype)

### Operace s tenzory
S tenzory je možné dělat nepřeberné množství operací [viz zde](https://pytorch.org/docs/stable/torch.html).

Zejména různé indexace, změny tvaru, skládání a matematické operace.

In [ ]:
tensor = torch.eye(4)
print(tensor)

print(f"První řádek: {tensor[0,:]}")
print(f"Druhý sloupec: {tensor[:, 1]}")
print(f"Poslední sloupec: {tensor[..., -1]}")

**Obsah tenzoru je měnitelný**

In [ ]:
tensor[:,1] = 10
print(tensor)

In [ ]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

### Kopírování tenzoru
Při kopírování je třeba použít funkci `.clone()` jinak získáme pouze nový ukazatel na původní tenzor.

In [ ]:
a = torch.tensor([[1, 2], [3,4]])
print(a)
b = a
b[0,0] = 10
print(b)
print(a)

print("\nSprávné kopírování")
a = torch.tensor([[1, 2], [3,4]])
b = a.clone()
b[0,0] = 10
print(b)
print(a)

#### Maticové násobení a násobení po složkách

In [ ]:
print(tensor)

print("\nMaticové násobení\n")
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose of a tensor
y1 = tensor @ tensor.T
print(y1)
y2 = tensor.matmul(tensor.T)
print(y2)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out = y3)
print(y3)
y4 = torch.matmul(tensor, tensor.T)
print(y4)


print("\nNásobení po složkách\n")
# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)
print(z1)
print(z2)

z3 = torch.mul(tensor, tensor)
print(z3)

In [ ]:
print(tensor)
print(tensor.sum())
print(tensor.sum(dim = 0))
print(tensor.sum(dim = 1))

### Převod do numpy array

In [ ]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

## Automatické počítání gradientu

Největší síla knihovny PyTorch je ale v možnosti atutomaticky počítat gradient podle složek tenzoru, u kterého si to při inicializaci zapneme pomocí parametru `requires_grad`.

Poté provedeme výpočet hodnot nějaké funkce a když na této funkci zavoláme `.backward()` dojde k výpočtu gradientu, který potom máme u daného tenzoru dostupný v property `grad`.

In [ ]:
# vyrobíme proměnnou (vécerozměrnou)
x = torch.tensor([1.0,2,3], requires_grad=True)
print(x)

# sestrojíme funkci, kterou chceme derivovat
y = torch.sum(torch.square(x))
print(y)

# provedeme zpětný chod
y.backward()

# výsledný gradient podle složek vstupní proměnné
print(x.grad)

### Úkol
Proveďte to samé pro funkci $\sin(x)$ a porovnejte se správným řešením.

In [ ]:
# vyrobíme proměnnou (vécerozměrnou)
x = torch.tensor([1.0,2,3], requires_grad=True)
print(x)

...

#### Pozor
Při opakovaném použití stejného tenzoru (proměnné) ve výpočtech a následném volání `.backward()` se gradient akumuluje - je tedy součtem všech gradientů získaných zpětným chodem.

V případě, že toto chování nechceme, je potřeba gradient resetovat na None.

In [ ]:
y = torch.sum(torch.sin(x))
# provedeme zpětný chod
y.backward()

# výsledný gradient podle složek vstupní proměnné
print(x.grad)

# Provnání se správným řešením
print(torch.cos(x))

In [ ]:
x.grad = None
y = torch.sum(torch.sin(x))
# provedeme zpětný chod
y.backward()

# výsledný gradient podle složek vstupní proměnné
print(x.grad)

# Provnání se správným řešením
print(torch.cos(x))

### Výpočty bez požadavku na gradient
Vidíme, že pokud tenzor `x` použijeme v nějakém dalším výpočtu - například v `torch.cos(x)`, ukazuje nám to u výsledku atribut `grad_fn`. 

Je to z důvodu, že je u `x` zapnuté _requires_grad = True_. Tedy u všech výpočtů, kam `x` vstupuje, se automaticky sleduje a vytváří výpočetní graf, 
aby bylo možné později provést zpětný chod.

Pokud toto chování nechceme a nějaký konkrétní výpočet sledovat necheme (protože ho nebudeme chtít derivovat), musíme buď místo `x` použít `x.detach()` 
nebo výpočet provést ve `with` bloku s  `torch.no_grad()`.

In [ ]:
with torch.no_grad():
    print(torch.cos(x))

print(torch.cos(x.detach()))

# Model neuronové sítě v PyTorch

Nyní použijeme knihovnu PyTorch pro sestrojení modelu stejné neuronové sítě, jako jsme vytvářeli ručně na minulém cvičení.

### XOR data

Opět si připravíme data ve dvou variantách.

In [ ]:
# Příznaky
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
])

# Výstupní třída            
y = np.array([0, 1, 1, 0])

# Varianta s více body
Xm = []
ym = []
for i in range(len(y)):
    for e in np.random.randn(100, len(X[i])):
        Xm.append(X[i] + e * 0.1)
        ym.append(y[i])
Xm = np.array(Xm)
ym = np.array(ym)

In [ ]:
# funkce na zobrazování
def plot_X_y(X, y):
    color = { 0: 'blue', 1: 'red' }
    colors = [color[y[i]] for i in range(len(y))]
    plt.scatter(X[:, 0], X[:, 1], c=colors)
    
plt.figure(figsize=(6,3))
ax1 = plt.subplot(121)
plot_X_y(X, y)
ax2 = plt.subplot(122)
plot_X_y(Xm, ym)
plt.show()

### Model neuronové sítě

Síť sestrojíme z elementů, které odpovídají jednotlivým vrstvám, [torch.nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear), a aktivačním funkcím, `torch.nn.functional.relu` a `torch.nn.functional.relu` (viz [torch.nn.functional](https://pytorch.org/docs/stable/nn.functional.html#non-linear-activation-functions)).

Zároveň tento model vytvoříme jako podtřídu [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html), která zajistí automatické rozpoznání parametrů atd.

Pro začátek implementujeme dvě metody:
* `__init__` - konstruktor, kde definujeme všechny prvky, které obsahují parametry.
* `forward` - Dopředný chod - tj. funkci, která ze vstupů spočte výstup.

Pro pozdější použití ještě implmentujeme:
* `predict` - udělá finální predikce
* `predict_proba` - bude pouze wrapper na forward, kdy nebudeme trackovat gradient

In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        """
        Constructor - defines all elements with parameters
        """
        super().__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        """
        Calculation of the output of the network
        """
        ...
        return x
    
    def predict_proba(self, x):
        """
        Wrapper that returns the probabilities
        """
        dataX = ...
        return pred.numpy().flatten()
    
    def predict(self, x):
        """
        Final predictions
        """
        return ...
        

model = NeuralNetwork()
print(model)

### Spuštění modelu na datech
Model můžeme snadno **spustit** na data. V takovém případě ho prostě použijeme jako funkci.
Protože při tomto kroku nechceme sledovat gradient, uděláme to v kontextu `with torch.no_grad()`.

Než to ale provedeme, musíme si data uložit jako tenzory.

In [ ]:
dataX = torch.tensor(X).type(torch.float)
datay = torch.tensor(y).type(torch.long)

print("Vstup")
print("x =",dataX)

print("\nVýstup sítě")
with torch.no_grad():
    print(model(dataX))

### Parametry modelu

Můžeme se podívat na aktuální hodnoty parametrů modelu, které jsou nějak defaultně inicializovány.

In [ ]:
for name, param in model.named_parameters():
    print(name,"\n",param)

### Náhodná inicializace parametrů
Pokud nám defaultní inicializace nevyhovuje (obecně by neměla), může je re-inicializovat.

K tomu použijeme funkce v namespace [torch.nn.init](https://pytorch.org/docs/stable/nn.init.html).

Zároveň si nastavíme napevno náhodný seed.

In [ ]:
torch.manual_seed(1)
torch.nn.init.normal_(model.fc1.weight)
torch.nn.init.zeros_(model.fc1.bias)
torch.nn.init.normal_(model.fc2.weight)
torch.nn.init.zeros_(model.fc2.bias)

for param in model.parameters():
    print(param)

### Manuální nastavení parametrů

Parametry můžeme také nastavit manuálně.

In [ ]:
# model.fc1.weight.data = torch.tensor([[ 0.9184, -0.6461], [-0.5820,  1.5953]])
# model.fc1.bias.data = torch.tensor([-0.0075,  0.5449])
# model.fc2.weight.data = torch.tensor([[-0.4268, -0.2199]])
# model.fc2.bias.data = torch.tensor([0.4188])

for param in model.parameters():
    print(param)

### Příprava trénování

Připravíme si ztrátovou funkci a optimizer.
* Jako ztrátovou funkci použijeme [binární relativní entropii](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html).
* Jako optimizer použijeme obyčejný [SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html).

In [ ]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

### Trénování

Připravíme si funkci, která provede trénování v předem zvoleném počtu epoch (kroků).

In [ ]:
def train(X, y, model, epochs, loss_fn, optimizer, print_res = True):
    dataX = torch.tensor(X, dtype=torch.float)
    datay = torch.tensor(y, dtype=torch.float).reshape(-1,1)
    losses = []
    for i in range(epochs):
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # Compute prediction error (forward)
        pred = model(dataX)
        loss = loss_fn(pred, datay)

        # Backpropagation
        loss.backward()
        
        # Optimization step
        optimizer.step()
        
        # Save loss value - loss je tenzor s jedním prvkem. Voláním .item() toto jedno číslo získáme - bez toho aby se trackoval gradient.
        loss = loss.item()
        losses.append(loss)
        
        if print_res and (i % 1000 == 999):
            print(f" step {i+1} loss: {loss:.4f}")
        
    return losses
        
losses = train(Xm, ym, model, 5000, loss_fn, optimizer)

plt.plot(range(1,len(losses)+1),losses, '.')
plt.show()

### Predikce jednotlivých tříd a vizualizace hranice

Podívejme se, jak se model natrénoval.

In [ ]:
print('Predikce tříd:', model.predict(X))
print('Predikce tříd:', model.predict_proba(X))
print('Skutečné třídy:', y)

from sklearn.metrics import accuracy_score
acc = accuracy_score(ym, model.predict(Xm))
print(f"Výsledná přesnost na velkém datasetu: {acc}")

In [ ]:
plot_X_y(Xm, ym)
nx, ny = 250, 150
x_min, x_max = plt.xlim()
y_min, y_max = plt.ylim()
xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx), np.linspace(y_min, y_max, ny))
Z = 1 - model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.pcolormesh(
    xx, yy, Z, cmap="red_blue_classes", norm=colors.Normalize(0.0, 1.0), zorder=0
)
plt.contour(xx, yy, Z, [0.5], linewidths=2.0, colors="white")

plt.show()

### Důležitost inicializace vah

Podívejme se nyní, kdybychom znovu náhodně inicializovali váhy a opakovali trénování, co se stane.

Zkusíme to udělat 10x.

In [ ]:
torch.manual_seed(2)
for i in range(10):
    torch.nn.init.normal_(model.fc1.weight)
    torch.nn.init.zeros_(model.fc1.bias)
    torch.nn.init.normal_(model.fc2.weight)
    torch.nn.init.zeros_(model.fc2.bias)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    train(Xm, ym, model, 5000, loss_fn, optimizer, print_res = False)
    acc = accuracy_score(ym, model.predict(Xm))
    print(acc)

Vidíme, že se ve většině případů nedokáže natrénovat. Nejspíše uvízne okolo nějakých lokálních extrémů.

Podívejme se, že se to nezlepší ani když použijeme jinou náhodnou inicializaci a optimizer Adam.

In [ ]:
torch.manual_seed(2)
for i in range(10):
    ...

# Neuronová síť na reálných datech

Použijeme nyní neuronovou síť na dataset MNIST.
Vezmeme si ho z knihovny `torchvision` [zde](https://pytorch.org/vision/main/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST)

Pak ale uděláme úkrok stranou (to bychom nemuseli) a vezmeme si z něj data do numpy.

In [ ]:
import torchvision

MNIST_data = torchvision.datasets.MNIST('MNIST/', download=True, train=True, transform=torchvision.transforms.ToTensor())

source_images = MNIST_data.data.numpy()/255
source_labels = MNIST_data.targets.numpy()

print(source_images.shape, source_labels.shape)

Změníme rozměry pole s obrázkovými daty na standardní formát [N,C,H,W], kde
* N - je počet obrázků (datových bodů)
* C - je počet kanálů (u stupňu šedi pouze jeden, u RGB 3)
* H - počet pixelů na výšku
* W - počet pixelů na šířku

In [ ]:
source_images = source_images.reshape(-1,1, 28,28)
print(source_images.shape, source_labels.shape)

In [ ]:
figure = plt.figure(figsize=(5, 5))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    img = source_images[i,0,:,:]
    label = source_labels[i]
    figure.add_subplot(rows, cols, i)
    plt.title(f"{label}")
    plt.axis("off")
    plt.imshow(img, cmap="gray")
plt.show()

### Příprava datasetu pro PyTorch

Pro práci s daty při trénování, tj. vytváření dávek, iterování přes ně atd., se v knihovně `PyTorch` používají různé podtřídy [torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset).

Detailní tutoriál je [zde](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

V našem případě si data převedeme na tensory a použijeme `torch.utils.data.TensorDataset`.

In [ ]:
dataset = torch.utils.data.TensorDataset(torch.tensor(source_images, dtype = torch.float), torch.tensor(source_labels, dtype = torch.long))

print(f"Počet prvků datasetu: {len(dataset)}")

In [ ]:
# dataset[0]

### Přístup k bodům datasetu

In [ ]:
figure = plt.figure(figsize=(5, 5))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    img, label = dataset[i]
    figure.add_subplot(rows, cols, i)
    plt.title(f"{label}")
    plt.axis("off")
    plt.imshow(img[0,:,:], cmap="gray")
plt.show()

### Rozdělení dat na trénovací a validační

Využijeme `torch.utils.data.random_split` s konrétním seedem.

In [ ]:
generator1 = torch.Generator().manual_seed(42)
train_data, val_data = torch.utils.data.random_split(dataset, [0.8, 0.2], generator1)
print("The length of train data is:",len(train_data))
print("The length of val data is:",len(val_data))

### Data pro dávkové učení

Připravíme dávkovač, který je schopný při trénování dodávat dávky. Použijeme k tomu `torch.utils.data.DataLoader`.

In [ ]:
training_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

In [ ]:
dataiter = iter(training_loader)
images, labels = next(dataiter)
print(images.shape)
print(labels.shape)

## Model neuronové sítě

Vyrobíme klasickou dopřednou síť, která na výstupu bude mít 10 neuronů.
Jako ztrátovou funkci použijeme kategorickou relativní entropii v implementaci [torch.nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), která má softmax v sobě - takže na výstupu necháme prostě neurony bez aktivační funkce.

In [ ]:
class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 16)
        self.fco = nn.Linear(16, 10)

    def forward(self, x):
        ...
        return x

model = MyNet()

# zkusíme tam dát data
outputs = model(images)
print(images.shape, outputs.shape)

### Parametry a jejich počet

Projděme si jednotlivé parametry a jejich počty.

In [ ]:
num_params = 0
for name, param in model.named_parameters():
    print(f"{name}\n# {param.numel()}")
    num_params += param.numel()
    
print(f"\nCelkový počet parametrů: {num_params}")

### Ztrátová funkce


Použijeme kategorickou relativní entropii v implementaci [torch.nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

Tato funkce pro každou dávku vrací průměrnou chybu predikce.

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

loss = loss_fn(outputs, labels)
print('Total loss for this batch: {}'.format(loss.item()))

### Optimizer

Jako optimizer použijeme Adam.

In [ ]:
# Optimizers specified in the torch.optim package
optimizer = torch.optim.Adam(model.parameters())

### Trénování jedné epochy

Napíšeme funkci, která provede natrénování jedné epochy modelu.

In [ ]:
def train_one_epoch(model, loss_fn, optimizer, training_loader):
    running_cum_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        ...

        # Gather data and report
        last_mean_loss = loss.item()
        running_cum_loss += last_mean_loss * inputs.shape[0]
        if i % 300 == 299:
            print(f"  batch {i+1} loss: {last_mean_loss:.3f}")
            
    # Return of the average over the whole training set
    return running_cum_loss / len(train_data)

loss = train_one_epoch(model, loss_fn, optimizer, training_loader)
print(f"Celková trénovací chyba: {loss}")

### Trénovací smyčka přes několik epoch

Připravíme si cyklus, ktere provede trénování v několika epochách. 

Na konci každé epochy si na validační množině spočteme: 
* průměrnou hodnotu ztrátové funkce,
* přesnost.

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
EPOCHS = 2

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))

    # One training step
    avg_loss = train_one_epoch(model, loss_fn, optimizer, training_loader)

    # Validation performance
    running_cum_vloss = 0.0
    vcorrect = 0
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        with torch.no_grad():
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
        running_cum_vloss += vloss * vinputs.shape[0]
        # count the correctly classified samoples
        vcorrect += (voutputs.argmax(1) == vlabels).float().sum()
    # Get average loss and accuraccy
    avg_vloss = running_cum_vloss / len(val_data)
    vacc = vcorrect / len(val_data)
    
    print(f"TRAIN loss: {avg_loss:.3f}, VALIDATION loss: {avg_vloss:.3f}, accuraccy: {vacc:.3f}")

### Ukázka výsledku na validačních datech

Podíváme se, jak model predikuje na první dávce validačních dat.

In [ ]:
dataiter = iter(validation_loader)
images, labels = next(dataiter)
print(images.shape)
print(labels.shape)

Použijeme funkci `argmax`, která na tenzoru vrátí index prvku, který nabývá maxima.

In [ ]:
with torch.no_grad():
    print(model(images).argmax(dim = 1))

print(labels.flatten())

### Predikce pravděpodobností

Pomocí funkce `softmax` transformujeme výstup sítě na pravděpodobnosti příslušnosti k jednotlivým třídám.

In [ ]:
with torch.no_grad():
    probabs = F.softmax(model(images), dim = 1)
    
print(probabs[0])